In [1]:
# Importaciones necesarias

try:
    import torch
    from torch.utils.data import DataLoader
    from torchvision import datasets
    from torchvision.transforms import ToTensor
    import torch.nn as nn
    import torch.nn.functional as F
except ImportError:
    !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
    import torch
    from torch.utils.data import DataLoader
    from torchvision import datasets
    from torchvision.transforms import ToTensor
    import torch.nn as nn
    import torch.nn.functional as F
    
try:
    from skimage.feature import hog
    from skimage.color import rgb2gray
except ImportError:
    !pip3 install scikit-image
    from skimage.feature import hog
    from skimage.color import rgb2gray

import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import confusion_matrix
import numpy as np
import random
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 MB 5.2 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [2]:
try:
    import torch
    from torch.utils.data import DataLoader
    from torchvision import datasets
    from torchvision.transforms import ToTensor
    import torch.nn as nn
    import torch.nn.functional as F
except ImportError:
    !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

try:
    from skimage.feature import hog
    from skimage.color import rgb2gray
except ImportError:
    !pip3 install scikit-image

import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import confusion_matrix
import numpy as np
import random
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

In [39]:
class MLP(nn.Module):
    def __init__(self, input_neurons, output_neurons, hidden_layers):
        super(MLP, self).__init__()
        self.input_neurons=input_neurons
        self.output_neurons=output_neurons
        self.hidden_layers=hidden_layers

        self.flatten = nn.Flatten()

        layers=[]
        prev_neurons=input_neurons
        if isinstance(hidden_layers,list):
            for i in hidden_layers:
                layers.append(nn.Linear(prev_neurons,i))
                layers.append(nn.ReLU())
                prev_neurons=i
        else:
            print("ERROR DE FORMATO, PASALE UNA LIST COMO HIDDENLAYERS")
        
        layers.append(nn.Linear(prev_neurons,output_neurons))

        self.hiddenlayer_stack = nn.Sequential(*layers)


    def forward(self, x):

        x=self.flatten(x)
        salida=self.hiddenlayer_stack(x)
        return salida

In [44]:
# Ejemplo de uso 

mlp = MLP(
    input_neurons=160*160*3,
    output_neurons=10,
    hidden_layers=[128,64]
)

def get_parameters(model):
    return sum(p.numel() for p in model.parameters())

print(f"O modelo ten {get_parameters(mlp):,} parámetros")

O modelo ten 9,839,434 parámetros


In [41]:
def train(dataloader, model, loss_fn, optimizer):
    # Obtemos o número total de lotes
    num_images = len(dataloader.dataset)
    # Poñemos o modelo en modo de adestramento
    model.train()
    # Iteramos sobre os lotes
    n_batch = 0
    for batch_imgs, batch_labels in dataloader:
        # Obtemos as predicións para o lote actual
        batch_predicted_probabilities = model(batch_imgs)
        # Calculamos a perda
        loss = loss_fn(batch_predicted_probabilities, batch_labels)
        # Poñemos a cero os gradientes dos parámetros do modelo
        optimizer.zero_grad()
        # Realizamos a retropropagación para calcular os gradientes de cada parámetro
        loss.backward()
        # Actualizamos os parámetros do modelo utilizando os gradientes calculados
        optimizer.step()

        # Cada 100 lotes, imprimimos a perda e o progreso
        if n_batch % 100 == 0:
            loss, imgs_processed = loss.item(), n_batch * len(batch_imgs)
            print(f"perda: {loss:>7f}  [{imgs_processed:>5d}/{num_images:>5d}]")

        n_batch += 1

    return loss

def test(dataloader, model):
    # Obtemos o número total de imaxes
    num_images = len(dataloader.dataset)
    # Poñemos o modelo en modo de avaliación (sen calcular gradientes)
    model.eval()
    # Inicializamos as variables para gardar a perda e a precisión
    test_loss, correct = 0, 0
    # Inicializamos as listas para gardar as clases predicidas e as clases reais
    test_predicted_classes, test_true_classes = [], []
    with torch.no_grad():
        for batch_imgs, batch_labels in dataloader:
            # Obtemos as predicións para o lote actual
            batch_predicted_probabilities = model(batch_imgs)
            # Calculamos a perda e acumulamos o valor
            test_loss += loss_fn(batch_predicted_probabilities, batch_labels).item()
            # Calculamos as clases predicidas (aquelas con maior probabilidade)
            batch_predicted_classes = batch_predicted_probabilities.argmax(dim=1)
            # Obtemos o número de predicións correctas e acumulamos o valor
            correct += (batch_predicted_classes == batch_labels).sum().item()
            # Gardamos as clases predicidas e as clases reais. Precisamos convertelas a listas de Python coa función tolist()
            test_predicted_classes.extend(batch_predicted_classes.tolist())
            test_true_classes.extend(batch_labels.tolist())
    # Calculamos a perda media e o porcentaxe de predicións correctas (accuracy), dividindo entre o número total de imaxes
    test_loss /= num_images
    correct /= num_images
    print(f"Perda media: {test_loss:>8f} Accuracy: {correct*100:>0.1f}%\n")

    return test_predicted_classes, test_true_classes

def val(dataloader, model):
    # Obtemos o número total de imaxes
    num_images = len(dataloader)
    # Poñemos o modelo en modo de avaliación (sen calcular gradientes)
    model.eval()
    # Inicializamos as variables para gardar a perda e a precisión
    test_loss, correct = 0, 0
    # Inicializamos as listas para gardar as clases predicidas e as clases reais
    test_predicted_classes, test_true_classes = [], []
    with torch.no_grad():
        for batch_imgs, batch_labels in dataloader:
            # Obtemos as predicións para o lote actual
            batch_predicted_probabilities = model(batch_imgs)
            # Calculamos a perda e acumulamos o valor
            test_loss += loss_fn(batch_predicted_probabilities, batch_labels).item()
            # Calculamos as clases predicidas (aquelas con maior probabilidade)
            batch_predicted_classes = batch_predicted_probabilities.argmax(dim=1)
            # Obtemos o número de predicións correctas e acumulamos o valor
            correct += (batch_predicted_classes == batch_labels).sum().item()
            # Gardamos as clases predicidas e as clases reais. Precisamos convertelas a listas de Python coa función tolist()
            test_predicted_classes.extend(batch_predicted_classes.tolist())
            test_true_classes.extend(batch_labels.tolist())
    # Calculamos a perda media e o porcentaxe de predicións correctas (accuracy), dividindo entre o número total de imaxes
    test_loss /= num_images
    correct /= num_images

    return test_loss

In [30]:
torchvision.datasets.Imagenette("dataset", download=True, size="160px")

100%|██████████████████████████████████████████████████████████████████████████████| 99.0M/99.0M [00:16<00:00, 6.06MB/s]


Dataset Imagenette
    Number of datapoints: 9469
    Root location: dataset

In [42]:
from torchvision import transforms

# Transformaciones básicas
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
])

dataset = ImageFolder(root="dataset/imagenette2-160/train", transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
tr_data, val_data = torch.utils.data.random_split(dataset, [train_size, val_size])
batch_size = 16
tr_dataloader = DataLoader(dataset=tr_data,
                              batch_size=batch_size,
                              shuffle=True)
val_dataloader = DataLoader(dataset=val_data,
                              batch_size=batch_size,
                              shuffle=True)

In [45]:
loss_fn = nn.CrossEntropyLoss()

learning_rate = 1e-3
optimizer = torch.optim.SGD(mlp.parameters(), lr=learning_rate) #Cambiar mlp por el modelo que se use

epochs = 20
patience = 4

train_losses = []
val_losses = []
best_val_loss = float('inf')
epochs_without_improvement = 0

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(tr_dataloader, mlp, loss_fn, optimizer)
    val_loss = val(val_dataloader, mlp)
    train_losses.append(train_loss.item())
    val_losses.append(val_loss)
    if val_loss < best_val_loss:
      best_val_loss = val_loss
      torch.save(mlp.state_dict(), "best_model.pth")
      epochs_without_improvement = 0
    else:
      epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
      print("Early Stopping")
      break

mlp.load_state_dict(torch.load("best_model.pth")) 
plt.figure(figsize=(8, 4))
plt.plot(train_losses, label='Perda de adestramento')
plt.plot(val_losses, label='Perda de validación')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.show()

test_predicted_classes, test_true_classes = test(test_dataloader, mlp)

Epoch 1
-------------------------------
perda: 2.288447  [    0/ 7575]
perda: 2.300019  [ 1600/ 7575]
perda: 2.190402  [ 3200/ 7575]


KeyboardInterrupt: 

## Arquitectura CNN modular

In [11]:
# Definición de la clase CNN
class CNN(nn.Module):

    # input_channels: número de canales de entrada 
    # conv_lavers: número de capas convolucionales
    # first_conv_filters: num filtros primera capa, se duplicarán en cada capa
    # con_kernel_size: tam filtros convolucionales (cuadrados)
    # batch_norm: booleano para usar normalización por lotes
    # hidden_mlp_layers: número de neuronas en la capa oculta de la red MLP
    # output_neurons: número de neuronas de salida
    def __init__(self, input_channels, conv_lavers, first_conv_filters, conv_kernel_size, batch_norm, hidden_mlp_layers, output_neurons):
        
        # Construtor 
        super(CNN, self).__init__()  

        # Arrays auxiliares
        layers = [] # Contiene las capas

        # Definición de capas según los parámetros
        for _ in range(conv_lavers): # Bucle para el número de capas convolucionades pasadas por parámetro
            layers.append(nn.Conv2d(in_channels=input_channels, out_channels=first_conv_filters, kernel_size=conv_kernel_size))
            if batch_norm: # Si se usa normalización por lotes, se añade la capa de normalización
                layers.append(nn.BatchNorm2d(first_conv_filters)) # Capa de normalización por lotes
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2, padding=0)) # Capa de max pooling
            layers.append(nn.ReLU()) # Capa de activación ReLU
            input_channels = first_conv_filters # Actualiza el número de canales de entrada para la siguiente capa convolucional
            first_conv_filters *= 2 # Duplica el número de filtros para la siguiente capa convolucional
        
        # Capa de convolución final
        self.conv = nn.Sequential(*layers) # Capa convolucional compuesta por las capas definidas en el bucle anterior
        
        # Capa de aplanamento
        self.flatten = nn.Flatten()

        # MLP
        layers_mlp = []
        prev_neurons = first_conv_filters
        if isinstance(hidden_mlp_layers,list):
            for i in hidden_mlp_layers:
                layers.append(nn.Linear(prev_neurons,i))
                layers.append(nn.ReLU())
                prev_neurons=i
        else:
            print("ERROR DE FORMATO, PASALE UNA LIST COMO HIDDENLAYERS")

        layers_mlp.append(nn.Linear(prev_neurons,output_neurons))
        
        self.mlp = nn.Sequential(*layers_mlp)

    def forward(self, x):

        x = self.conv(x)
        x = self.flatten(x)
        x = self.mlp(x)

        return x   

In [12]:
# Ejemplo de uso 

cnn = CNN(
    input_channels=1,
    conv_lavers=2,
    first_conv_filters=32,
    conv_kernel_size=3,
    batch_norm=True,
    hidden_mlp_layers=[128],
    output_neurons=10
)

def get_parameters(model):
    return sum(p.numel() for p in model.parameters())

print(f"O modelo ten {get_parameters(cnn):,} parámetros")

O modelo ten 20,298 parámetros


In [20]:
loss_fn = nn.CrossEntropyLoss()

learning_rate = 1e-3
optimizer = torch.optim.SGD(cnn.parameters(), lr=learning_rate)

epochs = 20

In [ ]:
batch_size = 8
train_dataloader = DataLoader(dataset=training_data,
                              batch_size=batch_size,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=batch_size,
                             shuffle=True)

In [24]:
train_losses_cnn = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(train_dataloader, cnn, loss_fn, optimizer)
    train_losses_cnn.append(train_loss.item())

test_predicted_classes_cnn, test_true_classes_cnn = test(test_dataloader, cnn)

Epoch 1
-------------------------------


NameError: name 'train' is not defined